# Stored Procedures

We have now reached the pinnacle of centralised and reusable coding within SQL Server and the T-SQL language, the Stored Procedure.

As with Functions, a Stored Procedure can accept parameters. However, that's where the similarities end.

A Stored Procedure can run anything from a single statement to numerous statements within a batch of code covering thousands of lines of T-SQL and with very few syntax restrictions.

It can return nothing, single values, tables, or even multiple tables at the same time.

You cannot join to a Stored Procedure, but it is possible to return data into a table and then join to that.

Stored Procedures also do not have inherent performance issues. The only performance issues with Stored Procedures are due to a developer's code as there are otherwise no performance issues specifically attributed to the Stored Procedure as an object.

Once executed, SQL Server stores the execution plan that it used to run the Stored Procedure and will use that same plan over and over going forwards. This tends to result in consistent performance and reduces the overhead of constant recompilation of code.

Also, crucially, we ARE allowed to create a temporary Stored Procedure in SQL Server by appending # to the procedure name in the same way that we do with Temporary Tables. This will allow us to complete a full suite of demos in this section.

#### Stored Procedure Syntax

A stored procedure is easy to code with the basic syntax for a Stored Procedure as follows:

<code>CREATE PROCEDURE <i>procName</i><br/>
(<br/>
&nbsp&nbsp&nbsp&nbsp@parameters parameterDataType<br/>
)<br/>
AS<br/>
&nbsp&nbsp&nbsp&nbsp<i>statements here</i><br/>
GO</code>

With this in mind we shall create our first Stored Procedure.

#### Single Value Stored Procedure

This will be a simple Procedure with no parameters (we do not have to use empty brackets as with a no parameter function) and will return a single value; the current date and time (as this is a temporary procedure, feel free to run the code below without fear of error):

In [ ]:
drop procedure if exists #basicTimeProc
go

create procedure #basicTimeProc
as
    select current_timestamp as currentDateTime
go

Okay, we now have our first Stored Procedure, but how do we run this?

#### Executing a Stored Procedure

This is a new syntax because we can return anything from a Stored Procedure and therefore we cannot treat it like a table.  In fact, the Stored Procedure must be executed:

In [ ]:
drop procedure if exists #basicTimeProc
go

create procedure #basicTimeProc
as
    select current_timestamp as currentDateTime
go

exec #basicTimeProc
go

It's as simple as that.

Let's play with some variants to see what they can offer.

#### Returning a Table

This is the next logical step, the returning a table of data.

We shall return the usual suspect columns from our hr.employee table alongside their job title and department (basically the same JOIN code that we used in our TVF and View sections):

In [ ]:
drop procedure if exists #basicEmployeeInfo
go

create procedure #basicEmployeeInfo
as
    select e.employeeID, e.title, e.firstName, e.lastName,
			concat_ws(' ', e.title, e.firstName, e.lastName) as fullname,
			t.jobTitle, d.department
	from JupyterDatabase.hr.employee as e
	join JupyterDatabase.hr.jobTitle as t
	on e.jobTitleID = t.jobTitleID
	join JupyterDatabase.hr.department as d
	on t.departmentID = d.departmentID
go

exec #basicEmployeeInfo
go

#### Passing a Parameter

As mentioned, a Stored Procedure is quite capable of receiving a parameter for use within the query.

We can demonstrate this using three basic examples.

In the first example we will pass in a @numDays offset to apply to our Date and Time value (note that we do not need brackets to pass in parameters):

In [ ]:
drop procedure if exists #parameterTimeProc
go

create procedure #parameterTimeProc
(
    @numDays int
)
as
    select dateadd(dd, @numDays, current_timestamp) as RequiredDateTime
go

exec #parameterTimeProc 1
go

In our second example we shall pass two parameters to the Stored Procedure and also demonstrate a slightly different syntax which allows us to specify exactly which value we are passing to which parameter.  If these are not specified then the values passed in are attributed to the parameters in order, therefore it can help you and others to be confident in your code by being explicit:

In [ ]:
drop procedure if exists #parameterTimeProc
go

create procedure #variableTimeProc
(
    @firstName varchar(25),
    @lastName varchar(35)
)
as
    select employeeID, firstName, lastName,
            concat_ws(' ', firstname, lastName) as fullName,
            salary, managerID
    from JupyterDatabase.hr.employee
    where firstname = @firstName
    and lastName = @lastName;
go

exec #variableTimeProc @firstName = 'Kevin', @lastName = 'Urquhart'
go

Lastly, we can also have Optional Parameters.  These are parameters that have default values attached to them and, as such, if we do not explicitly give them a value then they will use the default which we specify as part of the CREATE PROCEDURE script.

Before we write our script let's have a quick look at the sales figures for a couple of our sales people:

In [ ]:
select salesPersonID, sum(totalDue) SalesTotal, count(*) NumSales
from JupyterDatabase.sales.salesOrderHeader
where salesPersonID in (46, 47)
group by salesPersonID;

We can see from the output that SalesPersonID 46 has many more sales and can easily be separated from SalesPersonID 47 on either Sales Total or by Number of Sales.

Now, let's create a very specific and clearly rigged Stored Procedure that has three parameters; employeeID, salesTotal, and salesCount. We will make create it so that the latter two parameters have default values of 15 million and 80,000 respectively.

We shall then execute the Stored Procedure a total of 3 times with a variety of parameter options to become comfortable with the syntax around default parameters:

-   Only employeeID
-   EmployeeID and SalesTotal
-   All three parameters

First we shall create our Stored Procedure, including our three parameters, two of which will have default values assigned:

In [ ]:
drop procedure if exists #parameterSalesDefaults
go

create procedure #parameterSalesDefaults
(
    @employeeID int,
    @salesTotal decimal(15, 2) = 15000000,  -- Parameter created with a default value
	@salesCount int = 80000                 -- Parameter created with a default value
)
as
    select salesPersonID, sum(totalDue) as salesTotal,
			count(*) as salesCount
	from JupyterDatabase.sales.salesOrderHeader
	where salesPersonID = @employeeID       -- @employeeID used in the WHERE clause
	group by salesPersonID
	having sum(totalDue) > @salesTotal      -- @salesTotal used in the HAVING clause
	and count(*) > @salesCount;             -- @salesCount used in the HAVING clause
go

Now we need to call our Procedure with the varying parameter options stated above in order to investigate the syntax and validate the output.

Firstly we shall call the Stored Procedure with ONLY an EmployeeID and nothing else.  This means that we do not specify the other parameters and therefore SQL Server, upon not seeing them, will use the default values in the Stored Procedure definition.

Due to our carefully chosen defaults we expect this to return values for EmployeeID 46, but EmployeeID 47 has not made enough sales to satisfy the HAVING clause:

In [ ]:
drop procedure if exists #parameterSalesDefaults
go

create procedure #parameterSalesDefaults
(
    @employeeID int,
    @salesTotal decimal(15, 2) = 15000000,  -- Parameter created with a default value
	@salesCount int = 80000                 -- Parameter created with a default value
)
as
    select salesPersonID, sum(totalDue) as salesTotal,
			count(*) as salesCount
	from JupyterDatabase.sales.salesOrderHeader
	where salesPersonID = @employeeID       -- @employeeID used in the WHERE clause
	group by salesPersonID
	having sum(totalDue) > @salesTotal      -- @salesTotal used in the HAVING clause
	and count(*) > @salesCount;             -- @salesCount used in the HAVING clause
go

exec #parameterSalesDefaults @employeeID = 46;
exec #parameterSalesDefaults @employeeID = 47;
go

This runs successfully and we obtain the correct results.  EmployeeID 46 has their results returned, but EmployeeID 47 does not return as their results are lower than our default values.

Now let's override the defaults for SalesTotal with some very high numbers, ensuring that no-one has any results returned:

In [ ]:
drop procedure if exists #parameterSalesDefaults
go

create procedure #parameterSalesDefaults
(
    @employeeID int,
    @salesTotal decimal(15, 2) = 15000000,  -- Parameter created with a default value
	@salesCount int = 80000                 -- Parameter created with a default value
)
as
    select salesPersonID, sum(totalDue) as salesTotal,
			count(*) as salesCount
	from JupyterDatabase.sales.salesOrderHeader
	where salesPersonID = @employeeID       -- @employeeID used in the WHERE clause
	group by salesPersonID
	having sum(totalDue) > @salesTotal      -- @salesTotal used in the HAVING clause
	and count(*) > @salesCount;             -- @salesCount used in the HAVING clause
go

exec #parameterSalesDefaults @employeeID = 46, @salesTotal = 20000000;
exec #parameterSalesDefaults @employeeID = 47, @salesTotal = 20000000;
go

Again, this runs successfully and with the results we expect to see.

Lastly, we can execute the Stored Procedure using SalesTotal and SalesCount figures that are low, allowing both EmployeeIDs to return data:

In [ ]:
drop procedure if exists #parameterSalesDefaults
go

create procedure #parameterSalesDefaults
(
    @employeeID int,
    @salesTotal decimal(15, 2) = 15000000,  -- Parameter created with a default value
	@salesCount int = 80000                 -- Parameter created with a default value
)
as
    select salesPersonID, sum(totalDue) as salesTotal,
			count(*) as salesCount
	from JupyterDatabase.sales.salesOrderHeader
	where salesPersonID = @employeeID       -- @employeeID used in the WHERE clause
	group by salesPersonID
	having sum(totalDue) > @salesTotal      -- @salesTotal used in the HAVING clause
	and count(*) > @salesCount;             -- @salesCount used in the HAVING clause
go

exec #parameterSalesDefaults @employeeID = 46, @salesTotal = 10000, @salesCount = 100;
exec #parameterSalesDefaults @employeeID = 47, @salesTotal = 10000, @salesCount = 100;
go

As expected, both executions of our Stored Procedure now return data.

Hence we can see how easy it is to add default values to a Stored Procedure, to let SQL Server use these values, and how we can override them where necessary.

#### Returning Multiple Datasets

Maybe one result set is not enough and we wish to return several datasets at once.  This can easily be done by a Stored Procedure.

We wish to create a single Stored Procedure to return the following:
* The current date and time of execution
* The Employee Info about our passed in EmployeeID
* The Employee's Sales Summary (total Sales by Year)

This would be impossible with a Function or a View as each only allows one result set to be returned and in the form of a single table.

However, Stored Procedures have no such limitation as they can happily execute an entire batch at once, no matter how many statements that may contain.

Let's look at the above statements individually.

The current date and time of execution:

In [ ]:
select current_timestamp as executionTime;

Next the Employee Info about our passed in EmployeeID.  Written as a single, standalone statement we would utilise a variable, which we later turn into a parameter, as follows (we will also specify employeeID 46 for his adhoc example simply to make the code runnable):

In [ ]:
declare @employeeID int;

select @employeeID = 46;

select e.employeeID, e.title, e.firstName, e.lastName,
        concat_ws(' ', e.title, e.firstName, e.lastName) as fullname,
        t.jobTitle, d.department
from JupyterDatabase.hr.employee as e
join JupyterDatabase.hr.jobTitle as t
on e.jobTitleID = t.jobTitleID
join JupyterDatabase.hr.department as d
on t.departmentID = d.departmentID
where e.employeeID = @employeeID;

Finally, we wish to obtain the same EmployeeID's sales figures:

In [ ]:
declare @employeeID int;

select @employeeID = 46;

select salesPersonID, sum(totalDue) as salesTotal
from JupyterDatabase.sales.salesOrderHeader
where salesPersonID = @employeeID
group by salesPersonID;

Now we can piece all of these together into a single Stored Procedure:

In [ ]:
drop procedure if exists #multiTable
go

create procedure #multiTable
(
    @employeeID int
)
as
    select current_timestamp as executionTime;

    select e.employeeID, e.title, e.firstName, e.lastName,
            concat_ws(' ', e.title, e.firstName, e.lastName) as fullname,
            t.jobTitle, d.department
    from JupyterDatabase.hr.employee as e
    join JupyterDatabase.hr.jobTitle as t
    on e.jobTitleID = t.jobTitleID
    join JupyterDatabase.hr.department as d
    on t.departmentID = d.departmentID
    where e.employeeID = @employeeID;

    select salesPersonID, sum(totalDue) as salesTotal
    from JupyterDatabase.sales.salesOrderHeader
    where salesPersonID = @employeeID
    group by salesPersonID;
go

exec #multiTable 46
go

And as we required, this returns three tables of results.

As mentioned, we can have almost anything happen within a stored procedure.  We can DECLARE variables, SELECT data, JOIN tables, have multiple datasets etc. all in centralised and reusable code.

#### Capturing Outputs

Now that we can code and use Stored Procedures, what about capturing the output?

We may wish to use the results from a Stored Procedure for further processing or to retrieve values and place them into another Stored Procedure (for example, for logging purposes).

This, as you would expect, is possible although with one caveat, that being that in a T-SQL script we can only return and reuse one result set.  Other applications (for example those written in .net languages such as C# or VB) can read as many tables as we wish to return to them (aside from when testing, this is the most common usage for returning mutiple datasets from a Stored Procedure).

To do this we have differing methods whether you are returning a single value or a whole table.

#### Capturing Values

In order to achieve this we need to do the following:
* CREATE a Stored Procedure
 * Include an OUTPUT parameter
* DECLARE a variable in our T-SQL batch
* EXECUTE the Stored Procedure
 * Tell it to assign the output parameter to our variable

Then we can reuse the newly assigned variable for the remainder of our batch.

In code this looks as follows:

In [ ]:
---------------------------------------------------------------------------------------------------
-- Create our Stored Procedure
---------------------------------------------------------------------------------------------------

drop procedure if exists #salesTotal
go

create procedure #salesTotal
(
	@employeeID int,
	@salesTotal decimal(15, 2) output	-- We simply need add the 'output' keyword to our parameter
)
as
	select @salesTotal = sum(totalDue)		-- Assign our value to the variable
	from JupyterDatabase.sales.salesOrderHeader
	where salesPersonID = @employeeID;
go

---------------------------------------------------------------------------------------------------
---------------------------------------------------------------------------------------------------

-- Now we write our adhoc T-SQL code:

declare @salesAmount decimal(15, 2);		-- DECLARE our variable

exec #salesTotal 46, @salesAmount output;	-- Tell SQL Server to place the OUTPUT parameter into our variable

select @salesAmount as returnedOutput;		-- Use our variable as we please, in this case just display it
go

This appears a little more complex when seen, but actually when you follow through step by step you will see that it's actually quite logical and easy to follow and reproduce.

#### Capturing a Table

Capturing the data from a Stored Procedure into a Table is actually a little simpler than a single value.

In this circumstance we need to create a temporary table (or real table, but most likely a temporary one) which mimics the output results and then we simply tell our Stored Procedure to Insert its output into that table.

Let's break this down into sections to better understand it.

First we'll consider the following Stored Procedure:

In [ ]:
drop procedure if exists #allSalesData
go

create procedure #allSalesData
as
    select salesPersonID, sum(totalDue) as salesTotal,
			count(*) as salesCount
	from JupyterDatabase.sales.salesOrderHeader
	group by salesPersonID;
go

exec #allSalesData;
go

Looking at this output we can see that we need the following table in order to store this information:

In [ ]:
drop table if exists #allSalesDataTable
go

create table #allSalesDataTable
(
    salesPersonID int,
    salesTotal decimal(15, 2),
    salesCount int
)
go

Now we can combine these two into one set of statements:

In [ ]:
---------------------------------------------------------------------------------------------------
-- Create our Stored Procedure
---------------------------------------------------------------------------------------------------

drop procedure if exists #allSalesData
go

create procedure #allSalesData
as
    select salesPersonID, sum(totalDue) as salesTotal,
			count(*) as salesCount
	from JupyterDatabase.sales.salesOrderHeader
	group by salesPersonID;
go

---------------------------------------------------------------------------------------------------
---------------------------------------------------------------------------------------------------

-- Create our Temporary table to hold our data
drop table if exists #allSalesDataTable
go

create table #allSalesDataTable
(
    salesPersonID int,
    salesTotal decimal(15, 2),
    salesCount int
)
go

-- Execute the Proc, telling it to place that data into our Temporary Table
insert into #allSalesDataTable
(
    salesPersonID, salesTotal, salesCount
)
exec #allSalesData;
go

-- Now we can use this data:
select *
from #allSalesDataTable
order by salesTotal desc;
go